## Imports

### Imports & Stuff

In [1]:
import pandas as pd
## pandas 1.1.1 necessary
import numpy as np
import string 
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import math
from gensim.models import Word2Vec
from plotly.graph_objects import Scatter3d

In [2]:
!pwd

/home/conte/code/kiddbenji/delphes/notebooks/kiddbenji_notebooks


In [4]:
#Getting the merger data
merger = pd.read_pickle("../../raw_data/merger")

In [6]:
#Getting all the data
data = pd.read_pickle("../../raw_data/update_age_sex")
data = data.reset_index()

## Cleaning

In [7]:
#reducing the dataset
extract = merger[["index","content_y"]]

### Dropping the None

In [8]:
#identifying the None rows
count = 0
dropables = []
for i in extract["content_y"]:
    if i == None:
        dropables.append(count)
        count+= 1
    else:
        count+=1

In [9]:
#removing the Nones
extract = extract.drop(labels=dropables)

In [10]:
extract = extract.reset_index()

### Creating a Dictionary with Averaged Arrays

### Deleting Floats and Other Falses

In [12]:
### Removing the Falses within the Sentences
floats = []
for i,j in dict_tweets.items(): 
    if type(j) != np.ndarray:
        floats.append(i)

In [13]:
for i in floats:
    del dict_tweets[i]

### Creating DF

In [14]:
d_i_tweets = pd.DataFrame(dict_tweets).transpose()

In [15]:
d_i_tweets = d_i_tweets.reset_index()

In [16]:
d_i_tweets.columns

Index(['index',       0,       1,       2,       3,       4,       5,       6,
             7,       8,
       ...
           290,     291,     292,     293,     294,     295,     296,     297,
           298,     299],
      dtype='object', length=301)

### Merging DFs

In [17]:
all_info = data.merge(d_i_tweets,on="index")

In [18]:
all_info.columns

Index([    'index',    'mep_id',      'name',   'country',     'group',
       'nat_group',   'twitter',   'content',       'age',       'sex',
       ...
               290,         291,         292,         293,         294,
               295,         296,         297,         298,         299],
      dtype='object', length=310)

## Label Encoding

### Country

In [19]:
def countries(values):
    if values == "Netherlands":
        return 0
    elif values == "Denmark":
        return 1
    else:
        return 2

In [20]:
all_info["countries"] = all_info["country"].apply(countries)

### European Group

In [21]:
def groups(values):
    if values == 'Group of the Greens/European Free Alliance':
        return 0
    elif values == 'Renew Europe Group':
        return 1
    elif values == "Group of the European People's Party (Christian Democrats)":
        return 2
    elif values == 'Group of the Progressive Alliance of Socialists and Democrats in the European Parliament':
        return 3
    elif values == 'European Conservatives and Reformists Group':
        return 4
    elif values == 'Group of the European United Left - Nordic Green Left':
        return 5
    elif values == 'Non-attached Members':
        return 6
    else:
        return 7

In [22]:
all_info["european_group"] = all_info["group"].apply(groups)

### National Party

In [23]:
parties_list = ['Socialistisk Folkeparti',
       'Volkspartij voor Vrijheid en Democratie',
       'Christen Democratisch Appèl', 'Partij van de Arbeid',
       'ChristenUnie', 'GroenLinks', 'Forum voor Democratie',
       'Socialdemokratiet', 'Partij voor de Dieren', 'Democraten 66',
       'Venstre, Danmarks Liberale Parti', 'Det Radikale Venstre',
       'GO Realisme & Daadkracht', 'Staatkundig Gereformeerde Partij',
       'Enhedslisten', 'Det Konservative Folkeparti']

def parties(values):
    for i in parties_list:
        if values == i:
            return parties_list.index(i)

In [24]:
all_info["nat_party_code"] = all_info["nat_group"].apply(parties)

### Age

In [25]:
def age_func(values):
    if values <= 30:
        return 0
    if (values > 30) & (values <= 40):
        return 1
    if (values > 40) & (values <=50):
        return 2
    if (values > 50) & (values <= 60):
        return 3
    if values > 60:
        return 4

In [26]:
all_info["age_code"] = all_info["age"].apply(age_func)


In [27]:
all_info["age_code"]

0        0
1        0
2        0
3        0
4        0
        ..
10687    3
10688    3
10689    3
10690    3
10691    3
Name: age_code, Length: 10692, dtype: int64

## Exporting Document

### Exporting Words Matrix

In [28]:
d_i_tweets.to_csv('d_i_tweets.tsv',sep="\t", index=False, header=False)

In [29]:
all_info.columns

Index([         'index',         'mep_id',           'name',        'country',
                'group',      'nat_group',        'twitter',        'content',
                  'age',            'sex',
       ...
                    294,              295,              296,              297,
                    298,              299,      'countries', 'european_group',
       'nat_party_code',       'age_code'],
      dtype='object', length=314)

### Exporting Labels

In [30]:
all_info[["countries","european_group","nat_party_code","sex","age_code"]].to_csv("multi_variables.tsv",sep="\t", index=False, header=True)

In [31]:
all_info

,index,mep_id,name,country,group,nat_group,twitter,content,age,sex,...,294,295,296,297,298,299,countries,european_group,nat_party_code,age_code
0,5076,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Hvornår agerer #Bagmandspolitiet på anmeldelse...,30,1.0,...,-0.008443,-0.042849,-0.014880,0.041051,0.023303,-0.012236,1,0,0,0
1,5079,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,"Amazonas brænder, og skov- og klimakrisen er a...",30,1.0,...,0.000622,-0.024272,-0.017961,0.043717,0.026246,-0.006339,1,0,0,0
2,5080,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Covid-19 vaccine makers lobby the EU for immun...,30,1.0,...,0.006047,-0.046107,-0.023379,0.041657,0.028609,-0.009291,1,0,0,0
3,5081,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,Den britiske økonomi er i knæ: 'De kommende må...,30,1.0,...,0.009990,-0.015177,-0.028269,0.005696,0.034646,0.007942,1,0,0,0
4,5082,28161,Margrete AUKEN,Denmark,Group of the Greens/European Free Alliance,Socialistisk Folkeparti,MargreteAuken,"Jeg bliver så fly forbandet vred, ked af det, ...",30,1.0,...,0.010506,-0.017543,-0.027306,0.027216,0.015570,0.010784,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,133994,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,A Green New Deal for Europe must include great...,58,1.0,...,0.009091,-0.043326,-0.025808,0.038204,0.031567,-0.006687,0,3,3,3
10688,133995,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,A more socially and environmentally sustainabl...,58,1.0,...,0.003003,-0.046619,-0.022752,0.059461,0.049146,-0.007352,0,3,3,3
10689,133997,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,Goed gesprek met @EUombudsman Emily O’Reilly o...,58,1.0,...,-0.000672,-0.035493,-0.011981,0.056631,0.027491,-0.012153,0,3,3,3
10690,133998,5392,Lara WOLTERS,Netherlands,Group of the Progressive Alliance of Socialist...,Partij van de Arbeid,larawoltersEU,@pieterzwaan Ja hoor! Graag naar mijn EP email.,58,1.0,...,0.037959,-0.042401,-0.030223,0.052912,0.042841,-0.019456,0,3,3,3


## k-Means

In [32]:
from sklearn.neighbors import KNeighborsClassifier


d_i_tweets.drop(columns='index', inplace=True)
d_i_tweets


,0,1,2,3,4,5,6,7,8,9,...,290,291,292,293,294,295,296,297,298,299
0,-0.016846,-0.011860,-0.042851,0.044163,-0.027453,0.022471,0.000556,-0.036933,-0.001978,0.050484,...,0.010223,-0.011244,0.009240,0.028963,-0.008443,-0.042849,-0.014880,0.041051,0.023303,-0.012236
1,-0.025288,-0.020077,-0.029960,0.033417,-0.022732,0.031891,0.014065,-0.054808,0.008609,0.072909,...,0.014924,-0.009034,0.017069,0.036944,0.000622,-0.024272,-0.017961,0.043717,0.026246,-0.006339
2,-0.026789,0.006986,-0.025454,0.034253,-0.019140,0.003891,-0.009987,-0.062290,-0.005792,0.039410,...,0.033633,-0.010781,0.009733,-0.002281,0.006047,-0.046107,-0.023379,0.041657,0.028609,-0.009291
3,-0.018487,-0.005300,-0.019736,0.044298,-0.020414,0.032369,0.003270,-0.053760,-0.012529,0.039304,...,0.008235,-0.011501,0.014615,0.038758,0.009990,-0.015177,-0.028269,0.005696,0.034646,0.007942
4,-0.010963,-0.039525,-0.020190,0.044142,-0.027780,0.032248,0.009840,-0.049892,0.001871,0.058652,...,0.018865,-0.011564,0.013905,0.023749,0.010506,-0.017543,-0.027306,0.027216,0.015570,0.010784
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10687,-0.029210,0.011155,-0.006377,0.046776,-0.025457,0.002079,0.006012,-0.028870,0.015737,0.046332,...,0.037759,-0.015325,0.009431,-0.009702,0.009091,-0.043326,-0.025808,0.038204,0.031567,-0.006687
10688,-0.041173,0.010350,-0.021792,0.045523,-0.020995,0.017644,0.002863,-0.055886,-0.019077,0.063391,...,0.017315,0.016870,0.006894,-0.020515,0.003003,-0.046619,-0.022752,0.059461,0.049146,-0.007352
10689,-0.030016,0.003368,-0.033410,0.023910,-0.019755,0.018780,0.005391,-0.040908,-0.017813,0.065019,...,0.022029,-0.017187,0.017818,-0.004375,-0.000672,-0.035493,-0.011981,0.056631,0.027491,-0.012153
10690,-0.006542,0.005297,-0.014884,0.020711,-0.029233,0.003898,-0.024561,-0.053010,-0.021391,0.050471,...,0.029530,0.007000,-0.000413,-0.044168,0.037959,-0.042401,-0.030223,0.052912,0.042841,-0.019456


In [ ]:
all_info.columns

In [ ]:
X = all_info[d_i_tweets.columns]
y = all_info["countries"]

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
neigh = KNeighborsClassifier()
k_grid = {'n_neighbors' : [2,5,10,20,30]}

In [ ]:
grid_search = GridSearchCV(neigh, k_grid, n_jobs=-1, scoring = 'accuracy', cv = 5)
grid_search.fit(X_train,y_train)

In [ ]:
score = grid_search.score(X_test, y_test)

In [ ]:
score

In [34]:
from sklearn.metrics import plot_confusion_matrix


## six_countries


In [36]:
six_countries = pd.read_pickle("../../raw_data/six_countries")

In [37]:
six_countries.head(1)


,index,mep_id,name,country,group,nat_group,twitter,content,age,sex,...,293,294,295,296,297,298,299,countries,european_group,age_code
0,836,125045,Clara AGUILERA,Spain,Group of the Progressive Alliance of Socialist...,Partido Socialista Obrero Español,ClaraAguilera7,"Clara Aguilera: ""El criterio científico debe p...",56,1.0,...,0.041642,-0.003032,-0.044886,-0.004864,0.038302,-0.009395,0.010062,0,2,3


In [38]:
X = six_countries[d_i_tweets.columns]
y = six_countries["nat_group"]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [40]:
neigh = KNeighborsClassifier()
k_grid = {'n_neighbors' : [2,5,10,20,30]}

In [41]:
grid_search = GridSearchCV(neigh, k_grid, n_jobs=-1, scoring = 'accuracy', cv = 5)
grid_search.fit(X_train,y_train)

KeyboardInterrupt: 

In [ ]:
score = grid_search.score(X_test, y_test)

In [ ]:
score

In [ ]:
plot_confusion_matrix(grid_search,X_test, y_test)

In [42]:
all_countries = pd.read_hdf('../../raw_data/full_mean_df_hdf.h5', 'h5')


In [43]:
all_countries

,mep_id,name,country,group,nat_group,twitter,content,lang,0,1,...,290,291,292,293,294,295,296,297,298,299
0,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,bardzo dziękuję niezależne od władzy wolne med...,polish,-0.012661,0.002554,...,0.007531,-0.041275,0.029452,0.030989,0.010538,-0.049243,-0.025033,0.058314,0.019128,0.022892
1,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,komisja przyjęła arcyważne projekty pilotażowe...,polish,-0.018363,0.009202,...,0.010615,-0.039067,0.031028,0.025055,0.008500,-0.053187,-0.030836,0.053642,0.019144,0.015897
2,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,komisja przyjęła projekty pilotażowe mojego ws...,polish,-0.019916,0.006455,...,0.005200,-0.037808,0.030887,0.027038,0.009016,-0.054631,-0.028361,0.054603,0.024452,0.018427
3,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,w tym dniu w tym miejscu w tej godzinie proszę...,polish,-0.010152,0.005874,...,0.016721,-0.040829,0.033006,0.023900,0.013247,-0.052095,-0.029642,0.054064,0.021562,0.014025
4,197490,Magdalena ADAMOWICZ,Poland,Group of the European People's Party (Christia...,Independent,Adamowicz_Magda,bóg nie potrzebuje być przez nikogo broniony i...,polish,-0.017805,0.005459,...,0.012571,-0.039137,0.037016,0.028174,0.013037,-0.056093,-0.035508,0.053920,0.019459,0.015463
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137295,197652,Angelika WINZIG,Austria,Group of the European People's Party (Christia...,Österreichische Volkspartei,AngelikaWinzig,italiens budgetpolitik gefährdet nicht nur ita...,german,-0.032662,-0.020359,...,-0.006590,-0.010474,0.038501,0.017754,-0.031848,-0.114873,-0.063869,-0.001768,0.014983,0.010953
137296,197652,Angelika WINZIG,Austria,Group of the European People's Party (Christia...,Österreichische Volkspartei,AngelikaWinzig,danke martin für die organisation des eu somme...,german,-0.035984,-0.029438,...,-0.009227,-0.008789,0.036854,0.021920,-0.032733,-0.112913,-0.060042,-0.002471,0.018086,0.008367
137297,197652,Angelika WINZIG,Austria,Group of the European People's Party (Christia...,Österreichische Volkspartei,AngelikaWinzig,tadition amp moderne top betriebe netzwerktref...,german,-0.028948,-0.023442,...,-0.006545,-0.011271,0.042163,0.016308,-0.039746,-0.117086,-0.060991,-0.004830,0.018593,0.007534
137298,197652,Angelika WINZIG,Austria,Group of the European People's Party (Christia...,Österreichische Volkspartei,AngelikaWinzig,hier mein statement zur budgetrede von finanzm...,german,-0.034614,-0.024161,...,-0.010266,-0.012133,0.036686,0.015777,-0.028820,-0.107009,-0.058627,0.000139,0.018164,0.005121


In [44]:
all_countries.dropna(inplace=True)

In [45]:
X = all_countries[d_i_tweets.columns]
y = all_countries["name"]

In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [47]:
neigh = KNeighborsClassifier()
k_grid = {'n_neighbors' : [2,5,10,20,30]}

In [48]:
grid_search = GridSearchCV(neigh, k_grid, n_jobs=-1, scoring = 'accuracy', cv = 5)
grid_search.fit(X_train,y_train)

/home/conte/.pyenv/versions/3.7.7/envs/delphes/lib/python3.7/site-packages/sklearn/model_selection/_split.py:672: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [2, 5, 10, 20, 30]},
             scoring='accuracy')

In [49]:
score = grid_search.score(X_test, y_test)

In [50]:
score

0.10613003631333218

In [ ]:
plot_confusion_matrix(grid_search,X_test, y_test)